In [22]:
from __future__ import annotations

from collections import OrderedDict
from typing import Any, Dict, List, Optional, TypedDict, Union, cast

from pylabrobot.resources.coordinate import Coordinate
from pylabrobot.resources.container import Container
from pylabrobot.resources.deck import Deck
from pylabrobot.resources.itemized_resource import ItemizedResource
from pylabrobot.resources.resource import Resource
from pylabrobot.resources.resource_stack import ResourceStack
from pylabrobot.resources.tip_rack import TipRack, TipSpot
from pylabrobot.resources.trash import Trash
from pylabrobot.resources.utils import create_ordered_items_2d

物料类型构建

In [23]:

class ElectrodeSheetState(TypedDict):
    diameter: float  # 直径 (mm)
    thickness: float  # 厚度 (mm)
    mass: float  # 质量 (g)
    material_type: str  # 材料类型（正极、负极、隔膜、弹片、垫片、铝箔等）
    info: Optional[str]  # 附加信息

class ElectrodeSheet(Resource):
    """极片类 - 包含正负极片、隔膜、弹片、垫片、铝箔等所有片状材料"""

    def __init__(
        self,
        name: str = "极片",
        size_x=10,
        size_y=10,
        size_z=10,
        category: str = "electrode_sheet",
        model: Optional[str] = None,
    ):
        """初始化极片

        Args:
            name: 极片名称
            category: 类别
            model: 型号
        """
        super().__init__(
            name=name,
            size_x=size_x,
            size_y=size_y,
            size_z=size_z,
            category=category,
            model=model,
        )
        self._unilabos_state: ElectrodeSheetState = ElectrodeSheetState(
            diameter=14,
            thickness=0.1,
            mass=0.5,
            material_type="copper",
            info=None
        )

    # TODO: 这个还要不要？给self._unilabos_state赋值的？
    def load_state(self, state: Dict[str, Any]) -> None:
        """格式不变"""
        super().load_state(state)
        self._unilabos_state = state
    #序列化
    def serialize_state(self) -> Dict[str, Dict[str, Any]]:
        """格式不变"""
        data = super().serialize_state()
        data.update(self._unilabos_state)  # Container自身的信息，云端物料将保存这一data，本地也通过这里的data进行读写，当前类用来表示这个物料的长宽高大小的属性，而data（state用来表示物料的内容，细节等）
        return data


In [24]:
# TODO: 这个应该只能放一个极片
class MaterialHoleState(TypedDict):
    diameter: int
    depth: int
    max_sheets: int
    info: Optional[str]  # 附加信息

class MaterialHole(Resource):
    """料板洞位类"""
    children: List[ElectrodeSheet] = []

    def __init__(
        self,
        name: str,
        size_x: float,
        size_y: float,
        size_z: float,
        category: str = "material_hole",
        **kwargs
    ):
        super().__init__(
            name=name,
            size_x=size_x,
            size_y=size_y,
            size_z=size_z,
            category=category,
        )
        self._unilabos_state: MaterialHoleState = MaterialHoleState(
            diameter=20,
            depth=10,
            max_sheets=1,
            info=None
        )

    def get_all_sheet_info(self):
        info_list = []
        for sheet in self.children:
            info_list.append(sheet._unilabos_state["info"])
        return info_list
    
    #这个函数函数好像没用，一般不会集中赋值质量
    def set_all_sheet_mass(self):
        for sheet in self.children:
            sheet._unilabos_state["mass"] = 0.5  # 示例：设置质量为0.5g

    def load_state(self, state: Dict[str, Any]) -> None:
        """格式不变"""
        super().load_state(state)
        self._unilabos_state = state

    def serialize_state(self) -> Dict[str, Dict[str, Any]]:
        """格式不变"""
        data = super().serialize_state()
        data.update(self._unilabos_state)  # Container自身的信息，云端物料将保存这一data，本地也通过这里的data进行读写，当前类用来表示这个物料的长宽高大小的属性，而data（state用来表示物料的内容，细节等）
        return data
    #移动极片前先取出对象
    def get_sheet_with_name(self, name: str) -> Optional[ElectrodeSheet]:
        for sheet in self.children:
            if sheet.name == name:
                return sheet
        return None

    def has_electrode_sheet(self) -> bool:
        """检查洞位是否有极片"""
        return len(self.children) > 0

    def assign_child_resource(
        self,
        resource: ElectrodeSheet,
        location: Optional[Coordinate],
        reassign: bool = True,
    ):
        """放置极片"""
        # TODO: 这里要改，diameter找不到，加入._unilabos_state后应该没问题
        if resource._unilabos_state["diameter"] > self._unilabos_state["diameter"]:
            raise ValueError(f"极片直径 {resource._unilabos_state['diameter']} 超过洞位直径 {self._unilabos_state['diameter']}")
        if len(self.children) >= self._unilabos_state["max_sheets"]:
            raise ValueError(f"洞位已满，无法放置更多极片")
        super().assign_child_resource(resource, location, reassign)

    # 根据children的编号取物料对象。
    def get_electrode_sheet_info(self, index: int) -> ElectrodeSheet:
        return self.children[index]


#料板
class MaterialPlateState(TypedDict):
    hole_spacing_x: float
    hole_spacing_y: float
    hole_diameter: float
    info: Optional[str]  # 附加信息

class MaterialPlate(ItemizedResource[MaterialHole]):
    """料板类 - 4x4个洞位，每个洞位放1个极片"""
    
    children: List[MaterialHole]

    def __init__(
        self,
        name: str,
        size_x: float,
        size_y: float,
        size_z: float,
        ordered_items: Optional[Dict[str, MaterialHole]] = None,
        ordering: Optional[OrderedDict[str, str]] = None,
        category: str = "material_plate",
        model: Optional[str] = None,
        fill: bool = False
    ):
        """初始化料板

        Args:
            name: 料板名称
            size_x: 长度 (mm)
            size_y: 宽度 (mm)
            size_z: 高度 (mm)
            hole_diameter: 洞直径 (mm)
            hole_depth: 洞深度 (mm)
            hole_spacing_x: X方向洞位间距 (mm)
            hole_spacing_y: Y方向洞位间距 (mm)
            number: 编号
            category: 类别
            model: 型号
        """
        self._unilabos_state: MaterialPlateState = MaterialPlateState(
            hole_spacing_x=24.0,
            hole_spacing_y=24.0,
            hole_diameter=20.0,
            info="",
        )
        # 创建4x4的洞位
        # TODO: 这里要改，对应不同形状
        holes = create_ordered_items_2d(
            klass=MaterialHole,
            num_items_x=4,
            num_items_y=4,
            dx=(size_x - 4 * self._unilabos_state["hole_spacing_x"]) / 2,  # 居中
            dy=(size_y - 4 * self._unilabos_state["hole_spacing_y"]) / 2,  # 居中
            dz=size_z,
            item_dx=self._unilabos_state["hole_spacing_x"],
            item_dy=self._unilabos_state["hole_spacing_y"],
            size_x = 16,
            size_y = 16,
            size_z = 16,
        )
        if fill:
            super().__init__(
                name=name,
                size_x=size_x,
                size_y=size_y,
                size_z=size_z,
                ordered_items=holes,
                category=category,
                model=model,
            )
        else:
            super().__init__(
                name=name,
                size_x=size_x,
                size_y=size_y,
                size_z=size_z,
                ordered_items=ordered_items,
                ordering=ordering,
                category=category,
                model=model,
            )

    def update_locations(self):
        # TODO:调多次相加
        holes = create_ordered_items_2d(
            klass=MaterialHole,
            num_items_x=4,
            num_items_y=4,
            dx=(self._size_x - 3 * self._unilabos_state["hole_spacing_x"]) / 2,  # 居中
            dy=(self._size_y - 3 * self._unilabos_state["hole_spacing_y"]) / 2,  # 居中
            dz=self._size_z,
            item_dx=self._unilabos_state["hole_spacing_x"],
            item_dy=self._unilabos_state["hole_spacing_y"],
            size_x = 1,
            size_y = 1,
            size_z = 1,
        )
        for item, original_item in zip(holes.items(), self.children):
            original_item.location = item[1].location

In [25]:
class CoincellDeck(Deck):
    """纽扣电池组装工作站台面类"""

    def __init__(
        self,
        name: str = "coin_cell_deck",
        size_x: float = 1620.0,  # 3.66m
        size_y: float = 1270.0,  # 1.23m
        size_z: float = 500.0,
        origin: Coordinate = Coordinate(0, 0, 0),
        category: str = "coin_cell_deck",
    ):
        """初始化纽扣电池组装工作站台面

        Args:
            name: 台面名称
            size_x: 长度 (mm) - 3.66m
            size_y: 宽度 (mm) - 1.23m
            size_z: 高度 (mm)
            origin: 原点坐标
            category: 类别
        """
        super().__init__(
            name=name,
            size_x=size_x,
            size_y=size_y,
            size_z=size_z,
            origin=origin,
            category=category,
        )

In [7]:
import json

In [17]:
def upload_resources_to_unilab(wuliao: List[Resource]):
    from unilabos.resources.graphio import convert_resources_from_type
    from unilabos.config.config import BasicConfig 
    BasicConfig.ak = "beb0c15f-2279-46a1-aba5-00eaf89aef55"
    BasicConfig.sk = "15d4f25e-3512-4f9c-9bfb-43ab85e7b561"
    from unilabos.app.web.client import http_client
    resources = convert_resources_from_type(wuliao, [Resource])
    json.dump({"nodes": resources, "links": []}, open("button_battery_station_resources_unilab.json", "w"), indent=2)
    
    #print(resources)
    http_client.remote_addr = "https://uni-lab.test.bohrium.com/api/v1"
    
    http_client.resource_add(resources)

In [29]:
liaopan1 =  MaterialPlate(name="liaopan1", size_x=120.8, size_y=120.5, size_z=10.0, fill=True)

In [30]:
print(liaopan1)

MaterialPlate(name=liaopan1, size_x=120.8, size_y=120.5, size_z=10.0, location=None)


In [31]:
print(liaopan1.children)

[MaterialHole(name=liaopan1_materialhole_0_0, location=Coordinate(012.400, 084.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_0_1, location=Coordinate(012.400, 060.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_0_2, location=Coordinate(012.400, 036.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_0_3, location=Coordinate(012.400, 012.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_1_0, location=Coordinate(036.400, 084.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_1_1, location=Coordinate(036.400, 060.250, 010.000), size_x=16, size_y=16, size_z=16, category=material_hole), MaterialHole(name=liaopan1_materialhole_1_2, location=Coordinate(036.400, 036.250, 010.000), 

In [26]:
deck = CoincellDeck()
#创建一个4*4的物料板
liaopan1 =  MaterialPlate(name="liaopan1", size_x=120.8, size_y=120.5, size_z=10.0, fill=True)
#把物料板放到桌子上
deck.assign_child_resource(liaopan1, Coordinate(x=0, y=0, z=0))
#创建一个极片
for i in range(16):
    jipian = ElectrodeSheet(name=f"jipian_{i}", size_x= 12, size_y=12, size_z=0.1)
    liaopan1.children[i].assign_child_resource(jipian, location=None)
#创建一个4*4的物料板
liaopan2 =  MaterialPlate(name="liaopan2", size_x=120.8, size_y=120.5, size_z=10.0, fill=True)
#把物料板放到桌子上
deck.assign_child_resource(liaopan2, Coordinate(x=500, y=0, z=0))
#liaopan.children[3].assign_child_resource(jipian, location=None)
print(deck)

upload_resources_to_unilab([deck])

25-09-22 [15:15:08,950] [DEBUG] Starting new HTTPS connection (1): uni-lab.test.bohrium.com:443 [_new_conn:1049] [urllib3.connectionpool.connectionpool]


CoincellDeck(name=coin_cell_deck, location=Coordinate(000.000, 000.000, 000.000), size_x=1620.0, size_y=1270.0, size_z=500.0, category=coin_cell_deck)
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的

25-09-22 [15:15:09,218] [DEBUG] https://uni-lab.test.bohrium.com:443 "POST /api/v1/lab/material HTTP/1.1" 200 10 [_make_request:544] [urllib3.connectionpool.connectionpool]


In [20]:
liaopan1 = deck.get_resource("liaopan1")
print(liaopan1)

MaterialPlate(name=liaopan1, size_x=120.8, size_y=120.5, size_z=10.0, location=Coordinate(000.000, 000.000, 000.000))


In [ ]:
liaopan1 = deck.get_resource("liaopan1")


In [ ]:
print()

In [27]:
#在台面上找到料盘和极片
liaopan1 = deck.get_resource("liaopan1")
liaopan2 = deck.get_resource("liaopan2")
jipian1 = liaopan1.children[1].children[0]
#
print(jipian1)
#把物料解绑后放到另一盘上
jipian1.parent.unassign_child_resource(jipian1)
liaopan2.children[1].assign_child_resource(jipian1, location=None)
#print(jipian2.parent)
upload_resources_to_unilab([deck])

25-09-22 [15:17:44,322] [DEBUG] Starting new HTTPS connection (1): uni-lab.test.bohrium.com:443 [_new_conn:1049] [urllib3.connectionpool.connectionpool]


ElectrodeSheet(name=jipian_1, location=None, size_x=12, size_y=12, size_z=0.1, category=electrode_sheet)
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型

25-09-22 [15:17:44,599] [DEBUG] https://uni-lab.test.bohrium.com:443 "POST /api/v1/lab/material HTTP/1.1" 200 10 [_make_request:544] [urllib3.connectionpool.connectionpool]


25-09-22 [14:31:50,027] [DEBUG] Starting new HTTPS connection (1): uni-lab.test.bohrium.com:443 [_new_conn:1049] [urllib3.connectionpool.connectionpool]


转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot的时候，出现未知类型 material_hole
转换pylabrobot的时候，出现未知类型 electrode_sheet
转换pylabrobot

25-09-22 [14:31:50,358] [DEBUG] https://uni-lab.test.bohrium.com:443 "POST /api/v1/lab/material HTTP/1.1" 200 10 [_make_request:544] [urllib3.connectionpool.connectionpool]


<Response [200]>